In [1]:
# import necessary libraries

import openai
from openai import OpenAI

In [2]:
# instantiate client

client = OpenAI()

In [3]:
# Test: send a prompt 

response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{'role':'user', 'content':'what is elastic search'}]

    
)

In [4]:
response

ChatCompletion(id='chatcmpl-9hGFTbUYS43m5opP44FQztPzBrWDf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Elasticsearch is a distributed, RESTful search and analytics engine designed for horizontal scalability, reliability, and real-time search capabilities. It is built on top of the Apache Lucene search library and provides a simple and powerful API to store, index, search, and analyze large volumes of data in real-time. Elasticsearch is commonly used for log aggregation, full-text search, and analytics in a variety of applications and industries.', role='assistant', function_call=None, tool_calls=None))], created=1720097591, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=84, prompt_tokens=11, total_tokens=95))

In [5]:
response.choices[0].message.content

'Elasticsearch is a distributed, RESTful search and analytics engine designed for horizontal scalability, reliability, and real-time search capabilities. It is built on top of the Apache Lucene search library and provides a simple and powerful API to store, index, search, and analyze large volumes of data in real-time. Elasticsearch is commonly used for log aggregation, full-text search, and analytics in a variety of applications and industries.'

### Preparing the Document

In [18]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-07-03 12:54:55--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-03 12:54:55 (15.3 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [6]:
import minsearch

In [20]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-07-03 13:06:14--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-07-03 13:06:14 (38.5 MB/s) - ‘documents.json’ saved [658332/658332]



In [7]:
import json

In [8]:
# laod the json documents

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
# converting json file to a list

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Interacting with the Search Engine

In [13]:
# Create the Index and specify the search elements

index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields= ['course'] #condition to restrict documents to the course: data-engineering-zoomcamp
)

In [14]:
q = 'elasticsearch has quit unexpectedly'

In [17]:
index.fit(documents)

In [51]:
qu = 'can I still enroll if the course starts'

In [52]:
# when we think one of the text fields is more important than the other, we use boost
boost = {"question": 3.0, "section": 0.5}


results = index.search(
    query = qu,
    filter_dict = {"course":"data-engineering-zoomcamp"},
    boost_dict = boost,
    num_results = 5
)

In [53]:
results

[{'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ

## Generation

In [47]:
qu = 'can I still enroll if the course starts'

In [54]:
# Generic response from LLM
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role":"user", "content":qu}]

)
    

In [29]:
response.choices[0].message.content

'It ultimately depends on the policies and procedures of the specific institution or course provider. Some may allow late enrollment for a course that has already started, while others may not. It is best to contact the institution or course provider directly to inquire about their enrollment policies and see if it is possible to still enroll in the course.'

In [38]:
# Instantiate a prompt to guide the LLMs response
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ Database. 
use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT does not contain the answer, output NONE.

QUESTION : {question}

CONTEXT : {context}
""".strip()

In [55]:
context =""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"



In [56]:
print(context)

section: General course-related questions
question: Course - What can I do before the course starts?
answer: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anaconda)
Terraform
Git
Look over the prerequisites and syllabus to see if you are comfortable with these subjects.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I get support if I take the course in the self-paced mode?
answer: Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this

In [57]:
prompt = prompt_template.format(question = qu, context = context).strip()

In [58]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ Database. 
use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT does not contain the answer, output NONE.

QUESTION : can I still enroll if the course starts

CONTEXT : section: General course-related questions
question: Course - What can I do before the course starts?
answer: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anaconda)
Terraform
Git
Look over the prerequisites and syllabus to see if you are comfortable with these subjects.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: Gene

In [59]:
# Combining Both GPT and The Search result
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role":"user", "content":prompt}]

)
   
response.choices[0].message.content

'Yes, you can still join the course after the start date.'

# Cleaning up with a Function

In [93]:
# search the database

def search(query):
    boost = {"question": 3.0, "section": 0.5}


    results = index.search(
        query = query,
        filter_dict = {"course":"data-engineering-zoomcamp"},
        boost_dict = boost,
        num_results = 10
    )
    return results

In [94]:
search('do we really have to use github codespaces')

[{'text': "It's up to you which platform and environment you use for the course.\nGithub codespaces or GCP VM are just possible options, but you can do the entire course from your laptop.",
  'section': 'General course-related questions',
  'question': 'Environment - Do we really have to use GitHub codespaces? I already have PostgreSQL & Docker installed.',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Instruction on how to store secrets that will be avialable in GitHub  Codespaces.\nManaging your account-specific secrets for GitHub Codespaces - GitHub Docs',
  'section': 'Module 1: Docker and Terraform',
  'question': 'GitHub Codespaces - How to store secrets',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Choose the approach that aligns the most with your idea for the end project\nOne of those should suffice. However, BigQuery, which is part of GCP, will be used, so learning that is probably a better option. Or you can set up a local environment for most of this course.

In [189]:
# create the prompt template

def build_prompt(query, search_results):
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ Database. Do not explicitly use the word CONTEXT in your response. 
    use only the facts from the CONTEXT when answering the QUESTION.
    if the CONTEXT does not contain the answer, output NONE.
    
    QUESTION : {question}
    
    CONTEXT : {context}
    """.strip()

    context =""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question = query, context = context).strip()

    return prompt

In [96]:
# function to link query with openai-api

def llm(prompt):
    response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role":"user", "content":prompt}]
    
    )
       
    return response.choices[0].message.content

In [78]:
query = 'how do I install anaconda'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [79]:
answer

'To install Anaconda for this course, you can start by installing and setting up all the dependencies and requirements mentioned in the FAQ: Google cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, and Git.'

In [107]:
#query = 'how do I install anaconda'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [110]:
rag('What are the prerequisites for this course')

'NONE'

In [112]:
rag('How do I install the necessary dependencies to run the code?')

'To install the necessary dependencies to run the code for the course, you should start by installing and setting up the following:\n- Google cloud account\n- Google Cloud SDK\n- Python 3 (installed with Anaconda)\n- Terraform\n- Git\n\nMake sure to check the prerequisites and syllabus to ensure you are comfortable with these subjects.'

In [100]:
rag(query)

'To install Anaconda for the course, you can start by installing and setting up all the dependencies and requirements as listed in the FAQ Database, which includes Python 3 installed with Anaconda.'

In [109]:
rag('When can I expect to receive the confirmation email?')

'The confirmation email for the course will be sent on the 15th Jan 2024 at 17h00, which is when the course will officially start with the first "Office Hours" live.'

# Using Elastic Search

#command line.
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [124]:
from elasticsearch import Elasticsearch

In [125]:
es = Elasticsearch("http://localhost:9200")
#es.info()

In [ ]:
# create an index

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es.indices.create(index = index_name, body = index_settings)


In [127]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [129]:

for doc in tqdm(documents):
    es.index(index = index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 40.98it/s]


In [131]:
query = 'What are the prerequisites for this course'

In [132]:
# create the query

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [134]:
response_es = es.search(index= index_name, body = search_query)

In [141]:
response_docs = []

for hit in response_es['hits']['hits']:
    response_docs.append(hit['_source'])

In [143]:
response_docs

[{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou will probably also create your own repositories that host your notes, versions of your file, to do this. Here is a great tutorial that shows you how to do this: https://www.atlassian.com/git/tutorials/setting-up-a-repository\nRemember to ignore large database, .csv, and .gz files, and other files that should not be saved to a repository. Use .gitignore 

In [144]:
# put everything in query

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response_es = es.search(index= index_name, body = search_query)
    
    response_docs = []

    for hit in response_es['hits']['hits']:
        response_docs.append(hit['_source'])

    return response_docs


In [146]:
elastic_search('Could not establish connection to "MyServerName": Got bad result from install script')

[{'text': 'Change TO Socket\npgcli -h 127.0.0.1 -p 5432 -u root -d ny_taxi\npgcli -h 127.0.0.1 -p 5432 -u root -d ny_taxi',
  'section': 'Module 1: Docker and Terraform',
  'question': 'PGCLI - connection failed: :1), port 5432 failed: could not receive data from server: Connection refused could not send SSL negotiation packet: Connection refused',
  'course': 'data-engineering-zoomcamp'},
 {'text': "If you’re using an Anaconda installation:\nCd home/\nConda install gcc\nSource back to your RisingWave Venv - source .venv/bin/activate\nPip install psycopg2-binary\nPip install -r requirements.txt\nFor some reason this worked - the Conda base doesn’t have the GCC installed - (GNU Compiler Collection) a compiler system that supports various programming languages. Without this the it fails to install pyproject.toml-based projects\n“It's possible that in your specific environment, the gcc installation was required at the system level rather than within the virtual environment. This can happe

In [147]:
# rag using elastic_search 

def rag_es(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [149]:
rag_es('How do I enroll the course')



"To enroll in the course, you need to register before the course starts using the provided link. Additionally, make sure to subscribe to the course public Google Calendar (Desktop only) and join the course Telegram channel with announcements. Remember to also register in DataTalks.Club's Slack and join the channel."

## HW-01

In [157]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents_hw01 = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents_hw01.append(doc)

In [159]:
documents_hw01[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
# create an index

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es.indices.create(index = index_name, body = index_settings)


In [161]:
# applying the index to the document

for doc in tqdm(documents_hw01):
    es.index(index = index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 41.60it/s]


In [165]:
query_hw = "How do I execute a command in a running docker container?"

In [184]:
# create the query

search_query2 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_hw,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [185]:
response_hw = es.search(index= index_name, body = search_query2)

In [186]:
response_hw

ObjectApiResponse({'took': 12, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 690, 'relation': 'eq'}, 'max_score': 84.17781, 'hits': [{'_index': 'course-questions', '_id': 'XqFzfpAB2FQzjgIVumZY', '_score': 84.17781, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'EqGWfpAB2FQzjgIVwGrl', '_score': 84.17781, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run

In [187]:
response_docs_hw = []

for hit in response_hw['hits']['hits']:
    response_docs_hw.append(hit['_source'])

response_docs_hw

In [190]:
context =""
    
for doc in response_docs_hw:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
prompt = prompt_template.format(question = query, context = context).strip()


In [191]:
prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ Database. Do not explicitly use the word CONTEXT in your response. 
    use only the facts from the CONTEXT when answering the QUESTION.
    if the CONTEXT does not contain the answer, output NONE.
    
    QUESTION : {question}
    
    CONTEXT : {context}
    """.strip()

In [192]:
prompt_hw = prompt_template.format(question = query_hw, context = context).strip()

In [193]:
prompt_hw

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ Database. Do not explicitly use the word CONTEXT in your response. \n    use only the facts from the CONTEXT when answering the QUESTION.\n    if the CONTEXT does not contain the answer, output NONE.\n    \n    QUESTION : How do I execute a command in a running docker container?\n    \n    CONTEXT : section: 5. Deploying Machine Learning Models\nquestion: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nsection: 5. Deploying Machine Learning Models\nquestion: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so

In [196]:
len(prompt_hw)

1714

In [194]:
# Combining Both GPT and The Search result
answer_hw = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role":"user", "content":prompt_hw}]

)
   
answer_hw.choices[0].message.content

'To execute a command in a running docker container, you can use the `docker exec` command followed by the container id and the command you want to execute.'

In [195]:
len(answer_hw.choices[0].message.content)

155

In [197]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.8 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 19.8 MB/s eta 0:00:00m eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [200]:
import tiktoken

In [201]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [205]:
encoding.encode(prompt_hw)

[2675,
 2351,
 264,
 3388,
 12917,
 18328,
 13,
 22559,
 279,
 91022,
 3196,
 389,
 279,
 88436,
 505,
 279,
 32072,
 10199,
 13,
 3234,
 539,
 21650,
 1005,
 279,
 3492,
 88436,
 304,
 701,
 2077,
 13,
 720,
 262,
 1005,
 1193,
 279,
 13363,
 505,
 279,
 88436,
 994,
 36864,
 279,
 91022,
 627,
 262,
 422,
 279,
 88436,
 1587,
 539,
 6782,
 279,
 4320,
 11,
 2612,
 43969,
 627,
 1084,
 262,
 91022,
 551,
 2650,
 656,
 358,
 9203,
 264,
 3290,
 304,
 264,
 4401,
 27686,
 5593,
 5380,
 1084,
 262,
 88436,
 551,
 3857,
 25,
 220,
 20,
 13,
 71695,
 287,
 13257,
 21579,
 27972,
 198,
 7998,
 25,
 2650,
 656,
 358,
 7542,
 264,
 27686,
 5593,
 5380,
 9399,
 25,
 24083,
 279,
 5593,
 2217,
 304,
 21416,
 3941,
 323,
 68971,
 279,
 4441,
 2837,
 11,
 779,
 430,
 433,
 8638,
 264,
 28121,
 3290,
 627,
 29748,
 1629,
 482,
 275,
 1198,
 4177,
 2837,
 28121,
 366,
 1843,
 397,
 2746,
 279,
 5593,
 374,
 2736,
 4401,
 11,
 9203,
 264,
 3290,
 304,
 279,
 3230,
 5593,
 512,
 29748,
 4831,
 320,
 

In [203]:
len(encoding.encode(prompt_hw))

373

In [206]:
encoding.decode_single_token_bytes(2675)

b'You'